In [288]:
import pandas as pd
from sklearn.model_selection import train_test_split

train = pd.merge(pd.read_csv('training_set_features.csv'),
                 pd.read_csv('training_set_labels.csv'))
test = pd.read_csv('test_set_features.csv')
sample_submission = pd.read_csv('submission_format.csv')

train.shape, test.shape

((26707, 38), (26708, 36))

In [289]:
train.head()

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation,h1n1_vaccine,seasonal_vaccine
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN,0,0
1,1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe,0,1
2,2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo,0,0
3,3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN,0,1
4,4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb,0,0


In [290]:
# from pandas_profiling import ProfileReport
# profile = ProfileReport(train, minimal=True).to_notebook_iframe()
#
# profile

In [291]:
def wrangle(X):
    X = X.copy()

    X = X.drop(['respondent_id'], axis=1)

    return X

In [292]:
train = wrangle(train)
test = wrangle(test)


In [293]:
target = ['h1n1_vaccine', 'seasonal_vaccine']

train_features = train.drop(target, axis=1)

features = train_features.nunique().index.tolist()

In [294]:
y_train = train[target]
X_train = train[features]

X_test = test[features]

In [295]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.preprocessing import StandardScaler
import category_encoders as ce
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest
from xgboost import XGBClassifier
from imblearn.over_sampling import RandomOverSampler, SMOTE
from skopt import BayesSearchCV
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.impute import SimpleImputer
from sklearn.multioutput import MultiOutputClassifier

#When hyperparamater tuning, set tune to "True", and mark each model that we want to tune to "True".
tune = True
forest = True
xgboost = True

forest_distributions = {
    'model__n_estimators': [50, 100, 150, 200, 250, 300, 325, 350],
    'model__max_depth': [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30],
    'model__max_features': [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30,
                            31, 32, 33, 34, 35],
    'model__min_samples_leaf': [2, 3, 4, 5, 6, 7, 8]
}

xgboost_distributions = {
    'model__estimator__n_estimators': [125, 150, 175, 200, 225, 250, 260, 275, 300, 325, 350, 375, 400, 425, 450, 475, 500],
    'model__estimator__max_depth': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30],
    'model__estimator__learning_rate': [0.01, 0.03, 0.05, 0.07, 0.09, 0.10, 0.12, 0.14, 0.16, 0.18, 0.20, 0.22, 0.24, 0.26, 0.28, 0.30],
    'model__estimator__min_child_leaf':[1, 2, 3, 4, 5, 6],
    'model__estimator__min_child_weight': [1, 2, 3, 4, 5, 6],
    'model__estimator__colsample_bytree':[0.1, 0.15, 0.2, 0.3, 0.4, 0.50],
    'model__estimator__subsample':[0.6, 0.7, 0.8, 0.9, 1],
    'model__estimator__gamma':[0, 1, 2, 3, 4, 5],
    'model__estimator__scale_pos_weight': [20, 25, 30, 35, 40, 45, 50, 60]
 }

# forest_distributions = {
#     'model__n_estimators': [350, 375, 400, 425],
#     'model__max_depth': [29, 30, 31, 32, 33],
#     'model__max_features': [24, 25, 26, 27],
#     'model__min_samples_leaf': [4]
# }

# xgboost_distributions = {
#     'model__n_estimators': [260, 265, 270],
#     'model__max_depth': [26, 27],
#     'model__learning_rate': [0.08, 0.09],
#     'model__min_child_leaf':[3],
#     'model__min_child_weight': [4],
#     'model__colsample_bytree':[0.2],
#     'model__subsample':[0.9, 1],
#     'model__gamma':[0],
#     'model__scale_pos_weight': [25]
#   }

if tune: #If we are hyperparamater tuning, pass no parameters into estimators and find paramaters via GridSearchCV / RandomizedSearchCV

    import warnings
    warnings.filterwarnings('ignore') #ignore warnings generated by bayes search

    forest_pipeline = Pipeline([('encoder', ce.OrdinalEncoder()),
                                ('imputer', SimpleImputer(strategy='mean')),
                           ('model', RandomForestClassifier(random_state=1337))])

    xgboost_pipeline = Pipeline([('encoder', ce.OrdinalEncoder()),
                                 ('imputer', SimpleImputer(strategy='mean')),
                           ('model', MultiOutputClassifier(XGBClassifier(seed=1337)))])

    forest_search = BayesSearchCV(
        forest_pipeline,
        search_spaces=forest_distributions,
        n_iter=250,
        cv=5,
        scoring='roc_auc',
        random_state=1337,
        n_points=5,
        n_jobs=15
    )
    xgboost_search = BayesSearchCV(
        estimator=xgboost_pipeline,
        search_spaces=xgboost_distributions,
        n_iter=250,
        cv=5,
        scoring='roc_auc',
        random_state=1337,
        n_points=5,
        n_jobs=15
    )

    # forest_search = GridSearchCV(
    #     estimator=forest_pipeline,
    #     param_grid=forest_distributions,
    #     cv=3,
    #     verbose=3,
    #     scoring='f1_micro',
    #     n_jobs=15
    # )
    # xgboost_search = GridSearchCV(
    #     estimator=xgboost_pipeline,
    #     param_grid=xgboost_distributions,
    #     cv=3,
    #     verbose=3,
    #     scoring='f1_micro',
    #     n_jobs=15
    # )

    def report_step_forest(optim_result):
        score = forest_search.best_score_
        print("Random Forest best score: %s" % score)

    def report_step_xgboost(optim_result):
        score = xgboost_search.best_score_
        print("XGboost best score: %s" % score)

    #X_train, y_train = RandomOverSampler(sampling_strategy='not majority').fit_resample(X_train, y_train)

    if forest:
        forest_search.fit(X_train, y_train, callback=report_step_forest)
        forest_train_pred_proba = forest_search.predict_proba(X_train)
        forest_test_pred_proba = forest_search.predict_proba(X_test)

    if xgboost:
        xgboost_search.fit(X_train, y_train, callback=report_step_xgboost)
        xgboost_train_pred_proba = xgboost_search.predict_proba(X_train)
        xgboost_test_pred_proba = xgboost_search.predict_proba(X_test)

    #When hyperparameter tuning, pass our best estimators into votingclassifier. Only run when all 3 models are being tuned.
    # if forest and xgboost:
    #     voting_model = VotingClassifier(estimators=[('forest', forest_search.best_estimator_), #VotingClassifier is Soft Voting/Majority Rule classifier for unfitted estimators
    #                                                 ('xgboost', xgboost_search.best_estimator_),],
    #                                     voting='soft') #soft voting per recommendation from sklearn documentation, when used on tuned classifiers.
    #     voting_model_cv = voting_model
    #     voting_model.fit(X_train, y_train)
    #     combined_model_test_pred = voting_model.predict(X_test)

else: #If we are not hyperparameter tuning, pass in our best params(from previous tuning runs).
    forest_pipeline = Pipeline([('encoder', ce.OrdinalEncoder()),
                                ('imputer', SimpleImputer(strategy='mean')),
                            ('model', RandomForestClassifier(random_state=1337, n_jobs=14, max_depth=22,
                                                             min_samples_leaf=6, max_features=6, n_estimators=350))])

    xgboost_pipeline = Pipeline([('encoder', ce.OrdinalEncoder()),
                                ('model', XGBClassifier(seed=1337, n_jobs=15, colsample_bytree=0.3, gamma=3,
                                                        learning_rate=0.1, max_depth=3, min_child_leaf=1, min_child_weight=5,
                                                        n_estimators=375, scale_pos_weight=20, subsample=0.9))])


    #X_train, y_train = RandomOverSampler(sampling_strategy='minority').fit_resample(X_train, y_train) #over sample all but the majority class

    forest_pipeline_cv = forest_pipeline
    forest_pipeline.fit(X_train, y_train)
    forest_train_pred_proba = forest_pipeline.predict_proba(X_train)
    forest_test_pred_proba = forest_pipeline.predict_proba(X_test)

    # xgboost_pipeline_cv = xgboost_pipeline
    # xgboost_pipeline.fit(X_train, y_train)
    # xgboost_train_pred = xgboost_pipeline.predict(X_train)
    # xgboost_test_pred = xgboost_pipeline.predict(X_test)

    # voting_model = VotingClassifier(estimators=[('forest', forest_pipeline), #VotingClassifier is Soft Voting/Majority Rule classifier for unfitted estimators
    #                                             ('xgboost', xgboost_pipeline),],
    #                                 voting='soft') #soft voting per recommendation from sklearn documentation, when used on tuned classifiers.
    # voting_model_cv = voting_model
    # voting_model.fit(X_train, y_train)
    # combined_model_test_pred = voting_model.predict(X_test)


Random Forest best score: 0.8560847983771906
Random Forest best score: 0.8580955373682522
Random Forest best score: 0.8580955373682522
Random Forest best score: 0.8580955373682522
Random Forest best score: 0.8580955373682522
Random Forest best score: 0.8580955373682522
Random Forest best score: 0.8583970553639922
Random Forest best score: 0.8583970553639922
Random Forest best score: 0.8583970553639922
Random Forest best score: 0.8589689514934646
Random Forest best score: 0.8589689514934646
Random Forest best score: 0.8589689514934646
Random Forest best score: 0.8589689514934646
Random Forest best score: 0.8590210034413924
Random Forest best score: 0.8590210034413924
Random Forest best score: 0.8590210034413924
Random Forest best score: 0.8590210034413924
Random Forest best score: 0.8590210034413924
Random Forest best score: 0.8590210034413924
Random Forest best score: 0.8590210034413924
Random Forest best score: 0.8590210034413924
Random Forest best score: 0.8590210034413924
Random For

In [296]:
from sklearn.metrics import f1_score, classification_report, roc_auc_score
import numpy as np
if tune:
    if forest:
        print(forest_search.best_params_, '\n')
        print("Best Random Forest CV score: ", forest_search.best_score_, '\n')
        print('\n')

    if xgboost:
        print(xgboost_search.best_params_, '\n')
        print("Best xgboost CV score: ", xgboost_search.best_score_, '\n')
        print('\n')

    # if forest and xgboost:
    #     #print("Voting classifier, final F1-Micro score on validation set: ", f1_score(y_validation, combined_model_pred, average='micro'))
    #     print("Voting classifier, final F1-Micro cross-validation score: ", np.mean(cross_val_score(voting_model_cv, X_train,
    #                                                                                         y_train, scoring='f1_micro',
    #                                                                                         cv=3, n_jobs=15)))
else:
    # print("Random forest F1-Micro cross-validation score: ", np.mean(cross_val_score(forest_pipeline_cv, X_train,
    #                                                                          y_train, scoring='f1_micro', cv=3,
    #                                                                          n_jobs=15)), '\n')
    # print('\n')

    print("Random forest ROC AUC cross validation score: ", np.mean(cross_val_score(forest_pipeline_cv, X_train, y_train, scoring='roc_auc', cv=5,
                                                n_jobs=14)), '\n')


    #print('\n')

    #print("Random forest ROC AUC training score: ", roc_auc_score(y_train, forest_train_pred_proba))
    #
    # print("Voting classifier, final F1-Micro cross-validation score: ", np.mean(cross_val_score(voting_model_cv, X_train,
    #                                                                                         y_train, scoring='f1_micro',
    #                                                                                         cv=3, n_jobs=15)))

OrderedDict([('model__max_depth', 22), ('model__max_features', 6), ('model__min_samples_leaf', 6), ('model__n_estimators', 350)]) 

Best Random Forest CV score:  0.859039615733202 



OrderedDict([('model__estimator__colsample_bytree', 0.3), ('model__estimator__gamma', 3), ('model__estimator__learning_rate', 0.1), ('model__estimator__max_depth', 3), ('model__estimator__min_child_leaf', 1), ('model__estimator__min_child_weight', 5), ('model__estimator__n_estimators', 375), ('model__estimator__scale_pos_weight', 20), ('model__estimator__subsample', 0.9)]) 

Best xgboost CV score:  0.865659594959731 





In [297]:

# Make sure we have the rows in the same order
submission = sample_submission.copy()
np.testing.assert_array_equal(X_test.index.values,
                              submission.index.values)

# Save predictions to submission data frame
submission["h1n1_vaccine"] = forest_test_pred_proba[0][:, 1]
submission["seasonal_vaccine"] = forest_test_pred_proba[1][:, 1]
submission.to_csv('random_forest.csv', index=False)

submission = sample_submission.copy()
np.testing.assert_array_equal(X_test.index.values,
                              submission.index.values)

submission["h1n1_vaccine"] = xgboost_test_pred_proba[0][:, 1]
submission["seasonal_vaccine"] = xgboost_test_pred_proba[1][:, 1]
submission.to_csv('xgboost.csv', index=False)

In [298]:
submission.head()

,respondent_id,h1n1_vaccine,seasonal_vaccine
0,26707,0.748527,0.876982
1,26708,0.344693,0.294949
2,26709,0.802015,0.979315
3,26710,0.967905,0.992940
4,26711,0.929897,0.952205
